In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from script.gpt import GPT
from script.gpt_utils import *
from safetensors.torch import save_file

In [2]:
PATH = "checkpoints/checkpoint_final.pth"

# Preparation phase
chars, text, vocab_size = load_truyen_kieu_dataset("data/truyen_kieu.txt")
encoder, decoder = load_encoder_decoder(chars)

model = GPT(512, vocab_size, 512, 32, 6)
model.load_state_dict(torch.load(PATH, weights_only=True))
model.eval()

# Save model as safetensors
save_file(model.state_dict(), "checkpoints/truyen_kieu_gpt.safetensors")

Vocab size: 121
Number of characters: 101140


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
beautiful_print(model, decoder, 200, device=device)

Họa ở thanh mặt trời mù trước đTàng mới chơi
Thêm người năm người bước càng đẩy ngày
Trong không ghé giá ngoài đông ở trong
Thuyền dưới chịu phới phụ VLàng dù 
Một lời năn năm nhân mẫu chội trời
Họ hù

'\nHọa ở thanh mặt trời mù trước đTàng mới chơi\nThêm người năm người bước càng đẩy ngày\nTrong không ghé giá ngoài đông ở trong\nThuyền dưới chịu phới phụ VLàng dù \nMột lời năn năm nhân mẫu chội trời\nHọ hù'

In [11]:
def generate_text(model, num_sentences=4, device="cuda", temperature=1.0, top_k=None, block_size=128):
    # special_chars = ['\n']
    word_counts = [6, 8]  # Alternate between 6 and 8 words
    context = torch.zeros((1, 1), dtype=torch.long, device=device)
    sentences = []

    # Convert forbidden characters to token indices
    # special_char_indices = torch.tensor([encoder(ch) for ch in special_chars], device=device)

    model.eval()
    for i in range(num_sentences):
        idx = context
        word_count = word_counts[i % 2]  # Alternate sentence lengths
        char_count = 0
        current_word_count = 0
        sentence = ""

        while current_word_count < word_count:
            idx_cond = idx if idx.size(1) <= block_size else idx[:, -block_size:]
            logits, _ = model(idx_cond)
            logits = logits[:, -1, :] / temperature

            # logits[:, special_char_indices] = -float('Inf')
            
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            char = decoder([idx_next.item()])

            if char == "\n":
                continue

            # Count words based on spaces
            if char == " " and char_count > 1:
                current_word_count += 1

            sentence += char
            char_count += 1
            idx = torch.cat((idx, idx_next), dim=1)
            
        idx = torch.cat((idx, torch.zeros((1, 1), device='cuda:0', dtype=torch.long)), dim=1)
        sentences.append(sentence.strip())

    return sentences

In [14]:
generate_text(model)

['Thân công nhớ mới trông trở',
 'Chật bèo đâu vã lại còn ngơ nhớ',
 'Xem đau hãy Sở Khanh Bạc',
 'Bây mắt cớ người trong còn dưới ng']

In [6]:
# import re

# # Read original file
# with open('data/truyen_kieu_clean.txt', 'r', encoding='utf-8') as f:
#     text = f.read()

# # Remove special characters throughout the text
# text = re.sub(r"[:;!.,?\\'\"]", '', text)

# # Remove the last character if it's not a letter or number
# text = re.sub(r'[^a-zA-Z0-9]$', '', text)

# # Save to a new file
# with open('data/truyen_kieu.txt', 'w', encoding='utf-8') as f:
#     f.write(text)

# print("Cleaned text saved to 'data/truyen_kieu_clean_no_punct.txt'")